## Readme

Place this file in the ```_scripts``` folder within the transportation networks repo.   

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx
import pickle as p

In [2]:
# Assume you have not changed the relative paths after cloning the repository
root = os.path.dirname(os.path.abspath('.'))
root

'/Users/danielmckenzie/Google_Drive/Research/Fixed_Point_Networks/TransportationNetworks'

In [3]:
# This function imports trip data and then saves in Open MAtrix (OMX) format.
#
def import_matrix(matfile):
    f = open(matfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig=int(orig[0])

        d = [eval('{'+a.replace(';',',').replace(' ','') +'}') for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            # We map values to a index i-1, as Numpy is base 0
            mat[i, j] = matrix.get(i+1,{}).get(j+1,0)

    index = np.arange(zones) + 1

    myfile = omx.open_file('demand.omx','w')
    myfile['matrix'] = mat
    myfile.create_mapping('taz', index)
    myfile.close()

In [5]:
mod = os.path.join(root, 'SiouxFalls')
mod_files = os.listdir(mod)
source_file = os.path.join(mod, 'SiouxFalls_trips.tntp')
import_matrix(source_file) # import data and save as demand.omx

In [7]:
Trip_data = omx.open_file('demand.omx') # open the omx file
# A few nice commands to look at the metadata of an omx file
print('Shape:', Trip_data.shape())
print('Number of tables:', len(Trip_data))
print('Table names:', Trip_data.list_matrices())

Shape: (24, 24)
Number of tables: 1
Table names: ['matrix']


In [8]:
Trips = Trip_data['matrix']
# I'm pretty sure that NumOrigins = NumVertices for all traffic networks in this data set.
# For robustness, I'm keeping them separate for now. 
NumOrigins = Trip_data.shape()[0] 
NumVertices = 24 # Number of vertices in the graph
b_vectors = []
for i in range(NumOrigins):
    for j in range(NumOrigins):
        Amount_of_traffic = Trips[i,j]
        if Amount_of_traffic != 0.0:
            b_new = np.zeros(NumVertices)
            b_new[i] = -Amount_of_traffic
            b_new[j] = Amount_of_traffic
            b_vectors.append(b_new)

In [9]:
Trip_data.close() # Close the file

In [14]:
# Sanity check
print(b_vectors[3])
print('Number of b vectors is ' + str(len(b_vectors)))

[-200.    0.    0.    0.  200.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
Number of b vectors is 528


In [15]:
# Save b vectors into pickle format for easy transfer
p.dump(b_vectors, open("b_vectors.p", "wb"))

In [16]:
# Loading from a pickle file
b_vectors = p.load(open("b_vectors.p", "rb"))